In [1]:
import pickle
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [2]:
path = "/kaggle/input/dataset-wrist-filt/combined_wrist_filtered.pkl"

In [3]:
df_wrist = pd.read_pickle(path)

In [4]:
df_wrist.head()

,id,ACC_x,ACC_y,ACC_z,BVP,EDA,TEMP,label
214583,2.0,39.469502,27.912214,29.039982,-109.222518,1.637703,35.809887,1.0
214584,2.0,38.595181,27.796006,29.093324,-108.599068,1.637493,35.809884,1.0
214585,2.0,37.719606,27.683257,29.114913,-107.981064,1.637285,35.809882,1.0
214586,2.0,36.849917,27.575278,29.103982,-107.368689,1.637080,35.809879,1.0
214587,2.0,35.993018,27.473250,29.060322,-106.762018,1.636878,35.809876,1.0


In [5]:
df_wrist.isnull().sum()

id       0
ACC_x    0
ACC_y    0
ACC_z    0
BVP      0
EDA      0
TEMP     0
label    0
dtype: int64

In [7]:
df_wrist.isna().any()

id       False
ACC_x    False
ACC_y    False
ACC_z    False
BVP      False
EDA      False
TEMP     False
label    False
dtype: bool

In [8]:
df_wrist.groupby(['id', 'label']).count()

ACC_x   ACC_y   ACC_z     BVP     EDA    TEMP
id   label                                                
2.0  1.0    800800  800800  800800  800800  800800  800800
     2.0    430500  430500  430500  430500  430500  430500
     3.0    253400  253400  253400  253400  253400  253400
3.0  1.0    798000  798000  798000  798000  798000  798000
     2.0    448000  448000  448000  448000  448000  448000
     3.0    262500  262500  262500  262500  262500  262500
4.0  1.0    810601  810601  810601  810601  810601  810601
     2.0    444500  444500  444500  444500  444500  444500
     3.0    260400  260400  260400  260400  260400  260400
5.0  1.0    838600  838600  838600  838600  838600  838600
     2.0    451500  451500  451500  451500  451500  451500
     3.0    261800  261800  261800  261800  261800  261800
6.0  1.0    826000  826000  826000  826000  826000  826000
     2.0    455000  455000  455000  455000  455000  455000
     3.0    260400  260400  260400  260400  260400  260400
7.0  1.0    830200  830200  830200  830200  830200  830200
     2.0    448000  448000  448000  448000  448000  448000
     3.0    260401  260401  260401  260401  260401  260401
8.0  1.0    818300  818300  818300  818300  818300  818300
     2.0    469000  469000  469000  469000  469000  469000
     3.0    258999  258999  258999  258999  258999  258999
9.0  1.0    826000  826000  826000  826000  826000  826000
     2.0    451500  451500  451500  451500  451500  451500
     3.0    260400  260400  260400  260400  260400  260400
10.0 1.0    826000  826000  826000  826000  826000  826000
     2.0    507500  507500  507500  507500  507500  507500
     3.0    260400  260400  260400  260400  260400  260400
11.0 1.0    826000  826000  826000  826000  826000  826000
     2.0    476000  476000  476000  476000  476000  476000
     3.0    257600  257600  257600  257600  257600  257600
13.0 1.0    826001  826001  826001  826001  826001  826001
     2.0    464800  464800  464800  464800  464800  464800
     3.0    267400  267400  267400  267400  267400  267400
14.0 1.0    826000  826000  826000  826000  826000  826000
     2.0    472500  472500  472500  472500  472500  472500
     3.0    260401  260401  260401  260401  260401  260401
15.0 1.0    822500  822500  822500  822500  822500  822500
     2.0    480200  480200  480200  480200  480200  480200
     3.0    260400  260400  260400  260400  260400  260400
16.0 1.0    826000  826000  826000  826000  826000  826000
     2.0    471101  471101  471101  471101  471101  471101
     3.0    257600  257600  257600  257600  257600  257600
17.0 1.0    826700  826700  826700  826700  826700  826700
     2.0    506100  506100  506100  506100  506100  506100
     3.0    260400  260400  260400  260400  260400  260400

In [4]:
# Extraer features y labels
X = df_wrist[['ACC_x', 'ACC_y', 'ACC_z', 'BVP', 'EDA', 'TEMP']].values
y = df_wrist['label'].values
y.dtype

dtype('float64')

In [5]:
# Primero se converte la columna de tipo float a tipo categórico
y = pd.Categorical(y)
y.dtype

CategoricalDtype(categories=[1.0, 2.0, 3.0], ordered=False)

In [15]:
# Crear un modelo CNN para clasificación binaria para los datos recopilados de muñeca
# utilizando los parámetros que han salido mejores resultados de los datos de pecho
sampling_rate = 700  # Hz
window_time = 1  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

# Crear segmentos con overlapping windows
segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)

# Clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_segments_binary = np.where((y_segments == 1) | (y_segments == 3), 0, 1)

# Redimensionar X_segments para adaptar a la dimensión de entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# Definir el input layer para todos los sensores
input_layer = Input(shape=(sequence_length, num_features))

# Función para crear un bloque convolucional de 1D 
def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)
output = conv_block(output, filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_w_binary = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_w_binary.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 700, 6)]          0         
                                                                 
 conv1d (Conv1D)             (None, 343, 16)           1456      
                                                                 
 max_pooling1d (MaxPooling1  (None, 85, 16)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 76, 32)            5152      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 38, 32)            0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 1216)              0     

In [16]:
# Normalización de datos
scaler = StandardScaler()
X_segments_normalized = scaler.fit_transform(X_segments_reshaped.reshape(-1, num_features)).reshape(X_segments_reshaped.shape)

# Dividir los datos para entrenamiento(80%) y para prueba(20%)
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_binary, test_size=0.2, random_state=42)

# Ejecutar el modelo
optimizer = Adam(lr=0.01)
model_w_binary.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_w_binary.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_w_binary.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de muñeca: {accuracy * 100:.2f}%')

Epoch 1/40
1658/1658 [==============================] - 22s 12ms/step - loss: 0.1049 - accuracy: 0.9599 - val_loss: 0.0371 - val_accuracy: 0.9887
Epoch 2/40
1658/1658 [==============================] - 15s 9ms/step - loss: 0.0376 - accuracy: 0.9881 - val_loss: 0.0311 - val_accuracy: 0.9903
Epoch 3/40
1658/1658 [==============================] - 14s 9ms/step - loss: 0.0285 - accuracy: 0.9914 - val_loss: 0.0230 - val_accuracy: 0.9932
Epoch 4/40
1658/1658 [==============================] - 14s 9ms/step - loss: 0.0235 - accuracy: 0.9927 - val_loss: 0.0183 - val_accuracy: 0.9946
Epoch 5/40
1658/1658 [==============================] - 15s 9ms/step - loss: 0.0229 - accuracy: 0.9926 - val_loss: 0.0271 - val_accuracy: 0.9908
Epoch 6/40
1658/1658 [==============================] - 14s 8ms/step - loss: 0.0207 - accuracy: 0.9937 - val_loss: 0.0171 - val_accuracy: 0.9955
Epoch 7/40
1658/1658 [==============================] - 15s 9ms/step - loss: 0.0172 - accuracy: 0.9947 - val_loss: 0.0218 - val_a

In [18]:
y_pred_binary = (model_w_binary.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)

# Imprimir los resultados
print("Resultados del modelo de clasificación binaria para datos de muñeca: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 1s 3ms/step
Resultados del modelo de clasificación binaria para datos de muñeca: 
F1-score: 0.9931
Precisión: 0.9958
Recall: 0.9906
Matriz de confusión:
[[9194   18]
 [  38 4011]]


Ha salido buenos resultados con los parámetros del modelo para los datos de pecho, ahora se modifica otro parámetro batch size para intentar a mejorar más.

In [19]:
# Entrenar el modelo
model_w_binary.fit(X_train, y_train, epochs=40, batch_size=16, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_w_binary.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de muñeca: {accuracy * 100:.2f}%')

Epoch 1/40
3316/3316 [==============================] - 19s 6ms/step - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.0177 - val_accuracy: 0.9964
Epoch 2/40
3316/3316 [==============================] - 20s 6ms/step - loss: 0.0090 - accuracy: 0.9981 - val_loss: 0.0333 - val_accuracy: 0.9919
Epoch 3/40
3316/3316 [==============================] - 19s 6ms/step - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.0123 - val_accuracy: 0.9967
Epoch 4/40
3316/3316 [==============================] - 18s 5ms/step - loss: 0.0077 - accuracy: 0.9981 - val_loss: 0.0098 - val_accuracy: 0.9978
Epoch 5/40
3316/3316 [==============================] - 18s 5ms/step - loss: 0.0086 - accuracy: 0.9976 - val_loss: 0.0120 - val_accuracy: 0.9974
Epoch 6/40
3316/3316 [==============================] - 19s 6ms/step - loss: 0.0071 - accuracy: 0.9982 - val_loss: 0.0111 - val_accuracy: 0.9972
Epoch 7/40
3316/3316 [==============================] - 19s 6ms/step - loss: 0.0065 - accuracy: 0.9983 - val_loss: 0.0113 - val_ac

In [20]:
y_pred_binary = (model_w_binary.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)

# Imprimir los resultados
print("Resultados del modelo de clasificación binaria para datos de muñeca: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 1s 3ms/step
Resultados del modelo de clasificación binaria para datos de muñeca: 
F1-score: 0.9949
Precisión: 0.9969
Recall: 0.9970
Matriz de confusión:
[[9183   29]
 [  12 4037]]


Ha mejorado todavía más, estos resultados son muy bastantes buenos, ahora pasa a la clasificación de tres estados afectivos.

# Clasificación de tres estados afectivos de los datos de muñuca

In [20]:
# Generar un modelo CNN para la clasificaión de tres estados para los datos de muñeca
# Modificar el modelo anterior para adaptar a la nueva función
sampling_rate = 700  # Hz
window_time = 1  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)

# Redimensionar X_segments para la entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# One hot encoding, num_classes = 3
y_segments = y_segments - 1
y_segments_one_hot = to_categorical(y_segments, num_classes=3) 

# Dividir datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_segments_reshaped, y_segments_one_hot, test_size=0.2, random_state=42)

input_layer = Input(shape=(sequence_length, num_features))

def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)
output = conv_block(output, filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer para 3 clases, donde 3 es el num_classes
output_layer = Dense(3, activation='softmax')(x)

# Generar el modelo para clasificación de 3 estados
model_w_three = Model(inputs=input_layer, outputs=output_layer)

optimizer = Adam(lr=0.01)
model_w_three.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_w_three.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model_w_three.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de muñeca: {accuracy * 100:.2f}%')


Epoch 1/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.5822 - accuracy: 0.7634 - val_loss: 0.4503 - val_accuracy: 0.8033
Epoch 2/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.3677 - accuracy: 0.8565 - val_loss: 0.3634 - val_accuracy: 0.8574
Epoch 3/40
1658/1658 [==============================] - 10s 6ms/step - loss: 0.3057 - accuracy: 0.8837 - val_loss: 0.3081 - val_accuracy: 0.8893
Epoch 4/40
1658/1658 [==============================] - 12s 7ms/step - loss: 0.2788 - accuracy: 0.8938 - val_loss: 0.2694 - val_accuracy: 0.8906
Epoch 5/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.2712 - accuracy: 0.8973 - val_loss: 0.2601 - val_accuracy: 0.9050
Epoch 6/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.2492 - accuracy: 0.9053 - val_loss: 0.2215 - val_accuracy: 0.9202
Epoch 7/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.2325 - accuracy: 0.9105 - val_loss: 0.2186 - val_ac

In [21]:
y_pred_three = np.argmax(model_w_three.predict(X_test), axis=1)
y_true_three = np.argmax(y_test, axis=1)

f1 = f1_score(y_true_three, y_pred_three, average='weighted')
precision = precision_score(y_true_three, y_pred_three, average='weighted')
recall = recall_score(y_true_three, y_pred_three, average='weighted')

# Imprimir los reesultados
print("Resultados del modelo de clasificación de tres estados para datos de muñeca: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_three, y_pred_three)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 1s 2ms/step
Resultados del modelo de clasificación de tres estados para datos de muñeca: 
F1-score: 0.9591
Precisión: 0.9590
Recall: 0.9590
Matriz de confusión:
[[6707  111  173]
 [ 114 3916   19]
 [ 102   25 2094]]


Ahora se modifica el batch size a 16 como el modelo de clasificación binaria.

In [22]:
# Entrenar el modelo con batch size=16
model_w_three.fit(X_train, y_train, epochs=40, batch_size=16, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_w_three.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de muñeca: {accuracy * 100:.2f}%')

Epoch 1/40
3316/3316 [==============================] - 14s 4ms/step - loss: 0.1601 - accuracy: 0.9431 - val_loss: 0.1425 - val_accuracy: 0.9568
Epoch 2/40
3316/3316 [==============================] - 15s 5ms/step - loss: 0.1474 - accuracy: 0.9479 - val_loss: 0.2057 - val_accuracy: 0.9305
Epoch 3/40
3316/3316 [==============================] - 15s 5ms/step - loss: 0.1481 - accuracy: 0.9480 - val_loss: 0.1932 - val_accuracy: 0.9486
Epoch 4/40
3316/3316 [==============================] - 14s 4ms/step - loss: 0.1482 - accuracy: 0.9483 - val_loss: 0.1984 - val_accuracy: 0.9382
Epoch 5/40
3316/3316 [==============================] - 15s 5ms/step - loss: 0.1652 - accuracy: 0.9458 - val_loss: 0.1534 - val_accuracy: 0.9465
Epoch 6/40
3316/3316 [==============================] - 14s 4ms/step - loss: 0.1362 - accuracy: 0.9522 - val_loss: 0.1893 - val_accuracy: 0.9390
Epoch 7/40
3316/3316 [==============================] - 15s 5ms/step - loss: 0.1487 - accuracy: 0.9492 - val_loss: 0.1954 - val_ac

Ha empeorado los resultado, se modifica otros parámetros.

In [13]:
# Entrenar el modelo con lr=0,001
optimizer = Adam(lr=0.001)
model_w_three.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_w_three.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_w_three.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de muñeca: {accuracy * 100:.2f}%')

Epoch 1/40
1658/1658 [==============================] - 12s 7ms/step - loss: 0.1143 - accuracy: 0.9637 - val_loss: 0.1943 - val_accuracy: 0.9414
Epoch 2/40
1658/1658 [==============================] - 10s 6ms/step - loss: 0.1186 - accuracy: 0.9610 - val_loss: 0.1281 - val_accuracy: 0.9587
Epoch 3/40
1658/1658 [==============================] - 11s 7ms/step - loss: 0.1012 - accuracy: 0.9643 - val_loss: 0.1155 - val_accuracy: 0.9650
Epoch 4/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.1097 - accuracy: 0.9662 - val_loss: 0.2529 - val_accuracy: 0.9207
Epoch 5/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.1075 - accuracy: 0.9661 - val_loss: 0.1145 - val_accuracy: 0.9692
Epoch 6/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.0987 - accuracy: 0.9700 - val_loss: 0.1157 - val_accuracy: 0.9694
Epoch 7/40
1658/1658 [==============================] - 11s 6ms/step - loss: 0.1651 - accuracy: 0.9549 - val_loss: 0.1626 - val_ac

Ha empeorado todavía, se añade 2 capas de dropout para ver si mejoraría.

In [18]:
# Mejorar el modelo añadir 2 capas dropout
x = Dense(32, activation='relu')(Flatten()(output))
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer para 3 clases, donde 3 es el num_classes
output_layer = Dense(3, activation='softmax')(x)

# Generar el modelo para clasificación de 3 estados
model_w_three_2 = Model(inputs=input_layer, outputs=output_layer)

optimizer = Adam(lr=0.01)
model_w_three_2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_w_three_2.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model_w_three_2.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de muñeca: {accuracy * 100:.2f}%')


Epoch 1/20
1658/1658 [==============================] - 12s 7ms/step - loss: 6.3643 - accuracy: 0.5280 - val_loss: 1.0011 - val_accuracy: 0.5272
Epoch 2/20
1658/1658 [==============================] - 11s 6ms/step - loss: 0.9997 - accuracy: 0.5323 - val_loss: 0.9993 - val_accuracy: 0.5273
Epoch 3/20
1658/1658 [==============================] - 10s 6ms/step - loss: 0.9970 - accuracy: 0.5323 - val_loss: 0.9992 - val_accuracy: 0.5273
Epoch 4/20
1658/1658 [==============================] - 11s 7ms/step - loss: 0.9968 - accuracy: 0.5323 - val_loss: 0.9989 - val_accuracy: 0.5273
Epoch 5/20
1658/1658 [==============================] - 11s 6ms/step - loss: 1.0016 - accuracy: 0.5322 - val_loss: 0.9990 - val_accuracy: 0.5272
Epoch 6/20
1658/1658 [==============================] - 11s 6ms/step - loss: 1.0230 - accuracy: 0.5322 - val_loss: 0.9991 - val_accuracy: 0.5272
Epoch 7/20
1658/1658 [==============================] - 11s 6ms/step - loss: 0.9968 - accuracy: 0.5322 - val_loss: 0.9990 - val_ac

Ha empeorado mucho, así que añadir capa dropout no es el camino. Ahora comprueba con eliminir una capa de output.

In [19]:
# Aplicar una capa de output del bloque convolucional al input
output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer para 3 clases, donde 3 es el num_classes
output_layer = Dense(3, activation='softmax')(x)

# Generar el modelo para clasificación de 3 estados
model_w_three_3 = Model(inputs=input_layer, outputs=output_layer)

optimizer = Adam(lr=0.01)
model_w_three_3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_w_three_3.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model_w_three_3.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de muñeca: {accuracy * 100:.2f}%')


Epoch 1/40
1658/1658 [==============================] - 9s 5ms/step - loss: 0.6928 - accuracy: 0.7339 - val_loss: 0.6068 - val_accuracy: 0.7582
Epoch 2/40
1658/1658 [==============================] - 7s 5ms/step - loss: 0.4910 - accuracy: 0.8152 - val_loss: 0.4720 - val_accuracy: 0.8134
Epoch 3/40
1658/1658 [==============================] - 8s 5ms/step - loss: 0.4358 - accuracy: 0.8379 - val_loss: 0.4430 - val_accuracy: 0.8334
Epoch 4/40
1658/1658 [==============================] - 8s 5ms/step - loss: 0.4028 - accuracy: 0.8496 - val_loss: 0.3819 - val_accuracy: 0.8343
Epoch 5/40
1658/1658 [==============================] - 8s 5ms/step - loss: 0.3747 - accuracy: 0.8570 - val_loss: 0.3782 - val_accuracy: 0.8579
Epoch 6/40
1658/1658 [==============================] - 8s 5ms/step - loss: 0.3691 - accuracy: 0.8616 - val_loss: 0.3442 - val_accuracy: 0.8697
Epoch 7/40
1658/1658 [==============================] - 7s 4ms/step - loss: 0.3507 - accuracy: 0.8673 - val_loss: 0.3193 - val_accuracy:

No se ha mejorado más, los resultados conseguidos con el modelo inicial con los mismos parámetros que el modelo de datos de pecho son los mejores, es decir, con una precisión de 95,90%. 